In [190]:
import os
import pandas as pd
import folium
import datetime

## Introduction
This notebook presents different techniques on visualizing data using  Choropleth maps with [folium](https://python-visualization.github.io/folium/). Folium easily allows to manipulate data in Python and then visualize it in a Leaflet map.

Data used for this demo is property prices in Ireland and can be downloaded from [Property Price Register offical site](https://www.propertypriceregister.ie/)

The main goal was to use Folium Choropleth Map with a Time Slider that would show how average property prices have evolved month over month and what were the most expensive areas.

In [ ]:
Check working example: 

### Data Loading & Cleanup

Save data obtained from Propert Price Registrar as 'PPR-ALL.csv' and load data into dataframe.

In [196]:
df = pd.read_csv("PPR-ALL.csv", ',', encoding='utf-8')

In [197]:
df.shape

(369860, 9)

In [198]:
df.columns

Index(['Date of Sale (dd/mm/yyyy)', 'Address', 'Postal Code', 'County',
       'Price', 'Not Full Market Price', 'VAT Exclusive',
       'Description of Property', 'Property Size Description'],
      dtype='object')

Check empty values in each column

In [199]:
df.isnull().sum()

Date of Sale (dd/mm/yyyy)         0
Address                           0
Postal Code                  301195
County                            0
Price                             0
Not Full Market Price             0
VAT Exclusive                     0
Description of Property           0
Property Size Description    317380
dtype: int64

Format price column

In [200]:
ascii(df['Price'][0][0])

"'\\ufffd'"

In [201]:
df['Price'] = df['Price'].apply(lambda x : float(x.replace(u'\ufffd','').replace(',', '')))

In [202]:
#df.dropna(subset = ['Price'], inplace = True)

All properties are categorized via Description and Size. As for geolocation we only know the County of the property

In [203]:
df['Description of Property'].unique()

array(['Second-Hand Dwelling house /Apartment',
       'New Dwelling house /Apartment',
       'Second-Hannd Dwelling house /Apartment',
       'Teach/�ras�n C�naithe Ath�imhe', 'Teach/�ras�n C�naithe Nua',
       'Teach/?ras?n C?naithe Nua'], dtype=object)

In [204]:
def fix_description(desc):
    if 'Second-' in desc:
        return "second-hand house/apartment"
    elif 'Teach' in desc:
        return 'Teach/'
    else:
        return desc

Fix formatting of property description

In [205]:
df['Description of Property'] = df['Description of Property'].apply(lambda x : fix_description(x))

In [206]:
df['Property Size Description'].unique()

array([nan,
       'greater than or equal to 38 sq metres and less than 125 sq metres',
       'greater than 125 sq metres', 'less than 38 sq metres',
       'greater than or equal to 125 sq metres',
       'n�os m� n� n� cothrom le 38 m�adar cearnach agus n�os l� n� 125 m�adar cearnach',
       'n?os l? n? 38 m?adar cearnach'], dtype=object)

In [207]:
df['County'].unique()

array(['Dublin', 'Laois', 'Meath', 'Kilkenny', 'Limerick', 'Carlow',
       'Cork', 'Clare', 'Sligo', 'Cavan', 'Tipperary', 'Wicklow',
       'Roscommon', 'Wexford', 'Mayo', 'Donegal', 'Longford', 'Galway',
       'Offaly', 'Kildare', 'Waterford', 'Louth', 'Kerry', 'Westmeath',
       'Monaghan', 'Leitrim'], dtype=object)

DataFrame.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False, **kwargs)[source]¶

Format date column

In [208]:
df['date'] = df['Date of Sale (dd/mm/yyyy)'].apply (lambda x:  datetime.datetime.strptime(x,'%d/%m/%Y'))

In [210]:
df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)

In [211]:
df.head(5)

,Date of Sale (dd/mm/yyyy),Address,Postal Code,County,Price,Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,date,year,month
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,343000.0,No,No,second-hand house/apartment,NaN,2010-01-01,2010,1
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...,2010-01-03,2010,1
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,438500.0,No,No,second-hand house/apartment,NaN,2010-01-04,2010,1
3,04/01/2010,"1 The Haven, Mornington",NaN,Meath,400000.0,No,No,second-hand house/apartment,NaN,2010-01-04,2010,1
4,04/01/2010,"11 Melville Heights, Kilkenny",NaN,Kilkenny,160000.0,No,No,second-hand house/apartment,NaN,2010-01-04,2010,1


### Data insights

In [216]:
print ("Date range from: ", df['date'].min(), " to :", df['date'].max())

Date range from:  2010-01-01 00:00:00  to : 2019-06-14 00:00:00


Check number of data points and prices by county and property type

In [217]:
df[['County', 'Description of Property', 'Price']].groupby(['County', 'Description of Property']).aggregate(['count', 'min', 'max', 'sum', 'mean'])

Price                           \
                                         count        min           max   
County    Description of Property                                         
Carlow    New Dwelling house /Apartment    676   10000.00  2.097577e+06   
          second-hand house/apartment     3341    7500.00  1.350000e+06   
Cavan     New Dwelling house /Apartment    989    9000.00  1.300000e+06   
          second-hand house/apartment     4823    5412.00  2.104000e+06   
Clare     New Dwelling house /Apartment    922   14166.00  1.923952e+06   
          Teach/                             1  160000.00  1.600000e+05   
          second-hand house/apartment     7597    5987.00  3.318000e+07   
Cork      New Dwelling house /Apartment   6026    5925.00  7.828777e+06   
          Teach/                             2  175000.00  3.900000e+05   
          second-hand house/apartment    34401    5200.00  6.987348e+07   
Donegal   New Dwelling house /Apartment   1185   10000.00  2.904000e+06   
          Teach/                             2   74889.87  1.175000e+05   
          second-hand house/apartment     8938    5079.00  2.700000e+06   
Dublin    New Dwelling house /Apartment  20840    5963.00  6.920816e+07   
          Teach/                            17  115045.00  5.800000e+05   
          second-hand house/apartment    99806    5250.00  1.391650e+08   
Galway    New Dwelling house /Apartment   2260    6608.00  6.161485e+06   
          second-hand house/apartment    16672    5864.00  3.478100e+07   
Kerry     New Dwelling house /Apartment   1270    7000.00  2.925000e+06   
          second-hand house/apartment     9512    5250.00  2.700000e+06   
Kildare   New Dwelling house /Apartment   4553    9000.00  2.394950e+06   
          Teach/                             1  267000.00  2.670000e+05   
          second-hand house/apartment    13325    6500.00  2.650000e+07   
Kilkenny  New Dwelling house /Apartment    669   14380.00  2.125110e+06   
          second-hand house/apartment     5102    6000.00  5.141700e+06   
Laois     New Dwelling house /Apartment    947    9285.00  2.200000e+06   
          second-hand house/apartment     4635    6000.00  2.080000e+07   
Leitrim   New Dwelling house /Apartment    613   12000.00  8.400000e+05   
          second-hand house/apartment     2775    6000.00  8.150000e+05   
Limerick  New Dwelling house /Apartment   1430    6810.00  3.000000e+06   
...                                        ...        ...           ...   
Longford  second-hand house/apartment     2698    6847.31  7.350000e+05   
Louth     New Dwelling house /Apartment   1796   17621.00  6.013216e+06   
          Teach/                             1  179000.00  1.790000e+05   
          second-hand house/apartment     7952    6900.00  2.300000e+06   
Mayo      New Dwelling house /Apartment    842   14000.00  9.660000e+05   
          second-hand house/apartment     7920    5254.00  7.762438e+06   
Meath     New Dwelling house /Apartment   3638   14228.00  5.536500e+06   
          Teach/                             2  160000.00  3.696000e+05   
          second-hand house/apartment    10524    6000.00  4.000000e+06   
Monaghan  New Dwelling house /Apartment    483    7000.00  2.114520e+06   
          second-hand house/apartment     2447    5298.00  8.500000e+05   
Offaly    New Dwelling house /Apartment    418   20000.00  1.460000e+06   
          second-hand house/apartment     3970    6200.00  1.200000e+06   
Roscommon New Dwelling house /Apartment    710    5179.00  9.153656e+05   
          Teach/                             1  100000.00  1.000000e+05   
          second-hand house/apartment     4552    5500.00  1.075000e+06   
Sligo     New Dwelling house /Apartment    777   15000.00  5.559481e+06   
          second-hand house/apartment     4739    5177.00  1.900440e+06   
Tipperary New Dwelling house /Apartment    830    9900.00  3.000000e+06   
          second-hand house/apartment     8984    5080.00  6.60000

Select and prepare dataframe with data that will be used for Choropleth maps - only County, month, year and Avg Price is needed

In [218]:
data = df[['County', 'Description of Property', 'Price', 'year', 'month']]

In [219]:
data_agg = data.groupby(['County', 'Description of Property', 'year', 'month']).aggregate(['count', 'min', 'max', 'sum', 'mean'])

In [220]:
data_agg.columns = data_agg.columns.droplevel()

In [221]:
data_agg.reset_index(inplace=True)

In [222]:
data_agg.head()

,County,Description of Property,year,month,count,min,max,sum,mean
0,Carlow,New Dwelling house /Apartment,2010,1,4,127000.0,286343.61,821933.91,205483.4775
1,Carlow,New Dwelling house /Apartment,2010,2,3,132743.0,169162.98,451684.98,150561.6600
2,Carlow,New Dwelling house /Apartment,2010,3,2,132158.6,157709.24,289867.84,144933.9200
3,Carlow,New Dwelling house /Apartment,2010,4,2,170000.0,365000.00,535000.00,267500.0000
4,Carlow,New Dwelling house /Apartment,2010,5,6,56750.0,305000.00,971961.00,161993.5000


## Choropleth maps

Before making Choropleth with Time Slider, let's practice with 
[Choropleth maps](https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps).
Use data for selected month only and colour counties according to avg property price.
Use all Property types

In [223]:
by_county = data_agg[['County', 'year', 'month', 'mean']]

In [224]:
test = by_county[(by_county['year'] == 2010) &(by_county['month'] == 1)]

In [225]:
test = test[['County', 'mean']]

In [226]:
state_geo = f'ireland.json'

m = folium.Map(location=[53.305494, -7.737649], zoom_start=6)

folium.Choropleth(
    geo_data=state_geo,
    name='Average property prices in Jan 2010',
    data=test,
    columns=['County', 'mean'],
    key_on='feature.properties.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='yellow',
    legend_name='Average property prices',
).add_to(m)



In [227]:
#uncomment following line to display result
#m

Save result into html file

In [228]:
m.save(os.path.join('results', 'choropleth_mean_2010_01.html'))

#### Choropleth with self defined scale

Modyfying the legend is easy by passing your own bins for values and by using branca colorscale

In [229]:
by_county = data_agg[['County', 'year', 'month', 'sum']]

In [230]:
test = by_county[(by_county['year'] == 2010) &(by_county['month'] == 1)]

In [231]:
bins = list(test['sum'].quantile([0, 0.97, 0.98, 0.99, 1]))

In [232]:
bins

[212500.0,
 21184953.906299993,
 24577948.864199977,
 71234603.49700019,
 119657121.78999999]

In [233]:
m = folium.Map(location=[53.305494, -7.737649], zoom_start=6)

folium.Choropleth(
    geo_data=state_geo,
    name='Total spent on Properties in Jan 2010',
    data=test,
    columns=['County', 'sum'],
    key_on='feature.properties.id',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='yellow',
    legend_name='Total spent on Properties',
    bins=bins
).add_to(m)

folium.LayerControl().add_to(m)


In [234]:
#uncomment to display result
#m

In [235]:
m.save(os.path.join('results', 'choropleth_total_spent_2010_01.html'))

## Choropleth with Time Slider

[Source code of TimeSliderChoropleth](https://github.com/python-visualization/folium/blob/master/folium/plugins/time_slider_choropleth.py)

The class needs at least two arguments to be instantiated.

1. A string-serielized geojson containing all the features (i.e., the areas)
2. A style dictionary where the keys are the geojson feature ids and the values are dicts of `{time: style_options_dict}`:

`styledict = {
    '0': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        },
    ...,
    'n': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        }`

Good Time Slider Choropleth example notebook:
    https://github.com/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb

In [236]:
from folium import plugins
plugins.TimeSliderChoropleth

folium.plugins.time_slider_choropleth.TimeSliderChoropleth

### 1. Create geojson

Ireland counties Geojson data from https://github.com/deldersveld/topojson/blob/master/countries/ireland/ireland-counties.json

Load json data and transform to proper format

In [237]:
import json
with open('ireland.json') as json_file:
    geo_data = json.load(json_file)

In [238]:
geo_data['features'][0]

{'geometry': {'coordinates': [[[-6.592415, 52.708085],
    [-6.614788, 52.654218],
    [-6.71789, 52.633128],
    [-6.810019, 52.481218],
    [-6.916185, 52.446304],
    [-6.914688, 52.590519],
    [-6.975609, 52.653791],
    [-6.952951, 52.705377],
    [-7.090468, 52.732311],
    [-7.072441, 52.797934],
    [-7.065957, 52.822872],
    [-6.915828, 52.856717],
    [-6.735489, 52.884007],
    [-6.712332, 52.924335],
    [-6.463234, 52.893982],
    [-6.51247, 52.826791],
    [-6.631176, 52.831494],
    [-6.592415, 52.708085]]],
  'type': 'Polygon'},
 'properties': {'id': 'Carlow'},
 'type': 'Feature'}

Fix the format of the geojson, we need County name to be Extracted Properties and added as id

In [239]:
for county in geo_data['features']:
    county['id'] = county['properties']['id']

In [240]:
geo_data['features'][0]

{'geometry': {'coordinates': [[[-6.592415, 52.708085],
    [-6.614788, 52.654218],
    [-6.71789, 52.633128],
    [-6.810019, 52.481218],
    [-6.916185, 52.446304],
    [-6.914688, 52.590519],
    [-6.975609, 52.653791],
    [-6.952951, 52.705377],
    [-7.090468, 52.732311],
    [-7.072441, 52.797934],
    [-7.065957, 52.822872],
    [-6.915828, 52.856717],
    [-6.735489, 52.884007],
    [-6.712332, 52.924335],
    [-6.463234, 52.893982],
    [-6.51247, 52.826791],
    [-6.631176, 52.831494],
    [-6.592415, 52.708085]]],
  'type': 'Polygon'},
 'id': 'Carlow',
 'properties': {'id': 'Carlow'},
 'type': 'Feature'}

In [241]:
geo_data = json.dumps(geo_data)

### 2. Creating style dictionary

Style dictionary should be of following format:
    
    styledict = {
    '0': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        },
    ...,
    'n': {
        '2017-1-1': {'color': 'ffffff', 'opacity': 1}
        '2017-1-2': {'color': 'fffff0', 'opacity': 1}
        ...
        }
    
where 0...n will be the Counties name.
Each county will consist of data points for each month. Months will be stored as date using first day of month e.g. 2019-1-1.
Color will represent quantitive value of avg property price.


In [380]:
data = data_agg[['County', 'year', 'month', 'mean']].copy()

In [381]:
data.columns

Index(['County', 'year', 'month', 'mean'], dtype='object')

In [388]:
# Utility module for dealing with colormaps.
from branca.colormap import linear
cmap = linear.PuRd_09.scale(data['mean'].min(), data['mean'].max()/5)

In [389]:
data['mean'].min(), data['mean'].max()

(20000.0, 1962419.7233333334)

add color mapping representing value of avg price

In [358]:
#data['color'] = data['mean'].apply(cmap)

check color values

In [390]:
def norm(x):
    return (x - x.min()) / (x.max() - x.min())

Create style data dictionary

In [391]:
styledata = {}
#iterate through counties
for county in data['County'].unique():
    county_data = data[data['County'] == county]
    #define the 1st day of the month as epoch start and store it in proper format
    county_data['epoch'] = county_data.apply(lambda x : int(datetime.datetime(x['year'], x['month'], 1).timestamp()), axis=1)
    county_data['epoch']= county_data['epoch'].astype('U10')
    #use 0.8 as default opacity
    #county_data['opacity'] = norm(county_data['mean'])
    county_data['opacity'] = 0.8
    county_data['color'] = county_data['mean'].apply(cmap)
    county_data = county_data[['epoch', 'color', 'opacity']]
    county_data.set_index('epoch',inplace=True)
    styledata[county] = county_data

/home/magda/venv/pyvenv3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/magda/venv/pyvenv3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/magda/venv/pyvenv3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [392]:
styledata['Carlow'].head()

,color,opacity
epoch,,
1262300400,#df66b1,0.8
1264978800,#cb9bcb,0.8
1267398000,#cda0cd,0.8
1270072800,#df217a,0.8
1272664800,#ca92c6,0.8


In [393]:
styledict = {
    str(country): data.to_dict(orient='index') for
    country, data in styledata.items()
}

In [394]:
from folium.plugins import TimeSliderChoropleth
#tiles='Stamen Toner',
m = folium.Map(location=[53.305494, -7.737649],  zoom_start=7)

g = TimeSliderChoropleth(
    geo_data,
    styledict=styledict,
    name='Avg property price'

).add_to(m)

m.save(os.path.join('results', 'TimeSliderChoropleth.html'))